### **Curating body_back1b.h5ad**

Article:  Multi-scale spatial mapping of cell populations across anatomical sites in healthy human skin and basal cell carcinoma

DOI: https://doi.org/10.1101/2023.08.08.551504

Data Source : https://spatial-skin-atlas.cellgeni.sanger.ac.uk

##### **Mount farm**

mount-farm

##### **Packages required for curation**

In [1]:
#Import all packages required for curation

In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import scipy
from tqdm import tqdm
from scipy import sparse
from scipy.sparse import csr_matrix
import anndata as ad
import os
import subprocess
import math

### **Curation Schema**

##### **X (Matrix Layers)**

##### **AnnData object**

In [3]:
# Load the AnnData object

In [4]:
adata = sc.read_h5ad('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Data/WSSKNKCLsp12887264.h5ad')

In [5]:
# View the AnnData object

In [6]:
adata

AnnData object with n_obs × n_vars = 2546 × 17948
    obs: 'in_tissue', 'array_row', 'array_col', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes', 'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc', 'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes', 'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts', 'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC', 'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes', 'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1'
    uns: 'leiden', 'log1p', 'neighbors', 'pca', 'spatial', 'umap'
    obsm: 'X_spatial'

##### **X- expression matrix**

In [7]:
# View the expression matrix of the anndata object

In [8]:
adata.X

array([[-0.05748233, -0.05865315, -0.04291095, ..., -0.07650319,
        -0.2599088 , -0.13344628],
       [-0.05748233, -0.05865315, -0.04291095, ..., -0.07650319,
        -0.2599088 , -0.13344628],
       [-0.05748233, -0.05865315, -0.04291095, ..., -0.07650319,
        -0.2599088 , -0.13344628],
       ...,
       [-0.05748233, -0.05865315, -0.04291095, ..., -0.07650319,
        -0.2599088 , -0.13344628],
       [-0.05748233, -0.05865315, -0.04291095, ..., -0.07650319,
        -0.2599088 , -0.13344628],
       [-0.05748233, -0.05865315, -0.04291095, ..., -0.07650319,
        -0.2599088 , -0.13344628]], dtype=float32)

In [9]:
#data type of adata.X

In [10]:
type(adata.X)

numpy.ndarray

In [11]:
#convert to csr_matrix

In [12]:
adata.X = csr_matrix(adata.X)

In [13]:
adata.X

<2546x17948 sparse matrix of type '<class 'numpy.float32'>'
	with 45695608 stored elements in Compressed Sparse Row format>

In [14]:
# Print the matrix to check whether they are normalized counts or raw counts. if the matrix has floating numbers,they are normalized counts.if they are integers, they are raw counts.

In [15]:
print(adata.X)

  (0, 0)	-0.057482332
  (0, 1)	-0.058653146
  (0, 2)	-0.04291095
  (0, 3)	-0.11993884
  (0, 4)	-0.123748206
  (0, 5)	-0.048015893
  (0, 6)	-0.082888894
  (0, 7)	-0.071301796
  (0, 8)	-0.10044568
  (0, 9)	-0.3317558
  (0, 10)	-0.11245705
  (0, 11)	-0.19193329
  (0, 12)	-0.04264812
  (0, 13)	-0.11029607
  (0, 14)	-0.4108622
  (0, 15)	-0.2366933
  (0, 16)	-0.24423377
  (0, 17)	-0.08163287
  (0, 18)	6.84341
  (0, 19)	-0.1396072
  (0, 20)	-0.047535688
  (0, 21)	-0.21801063
  (0, 22)	-0.12587388
  (0, 23)	2.173744
  (0, 24)	-0.20097657
  :	:
  (2545, 17923)	-0.11786231
  (2545, 17924)	-0.027882816
  (2545, 17925)	-0.17116675
  (2545, 17926)	-0.046761286
  (2545, 17927)	-0.1966822
  (2545, 17928)	-0.24854457
  (2545, 17929)	-0.042491153
  (2545, 17930)	-0.24932185
  (2545, 17931)	-3.1958733
  (2545, 17932)	0.26208335
  (2545, 17933)	4.0706325
  (2545, 17934)	-2.9663908
  (2545, 17935)	0.22697276
  (2545, 17936)	0.9084899
  (2545, 17937)	2.0526142
  (2545, 17938)	-0.2146497
  (2545, 17939)	-1.

##### **Raw counts matrix**

In [16]:
# If X has normalized counts, check for the raw counts matrix.

In [17]:
#Here the raw counts are provided in a separate object, load the raw counts matrix

In [18]:
araw= sc.read_10x_h5('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Raw/all/WSSKNKCLsp12887264/filtered_feature_bc_matrix.h5')

/home/jovyan/my-conda-envs/shi_env/lib/python3.8/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [19]:
# view raw object

In [20]:
araw

AnnData object with n_obs × n_vars = 2546 × 36601
    var: 'gene_ids', 'feature_types', 'genome'

In [21]:
# view raw matrix

In [22]:
araw.X

<2546x36601 sparse matrix of type '<class 'numpy.float32'>'
	with 3269991 stored elements in Compressed Sparse Row format>

In [23]:
print(araw.X)

  (0, 36)	1.0
  (0, 44)	1.0
  (0, 59)	2.0
  (0, 60)	1.0
  (0, 99)	1.0
  (0, 173)	1.0
  (0, 261)	1.0
  (0, 283)	1.0
  (0, 291)	1.0
  (0, 429)	1.0
  (0, 436)	1.0
  (0, 441)	1.0
  (0, 487)	1.0
  (0, 524)	3.0
  (0, 593)	2.0
  (0, 601)	1.0
  (0, 611)	1.0
  (0, 640)	1.0
  (0, 796)	1.0
  (0, 874)	1.0
  (0, 926)	1.0
  (0, 943)	2.0
  (0, 975)	2.0
  (0, 1149)	1.0
  (0, 1405)	1.0
  :	:
  (2545, 35186)	1.0
  (2545, 35300)	1.0
  (2545, 35310)	1.0
  (2545, 35324)	1.0
  (2545, 35378)	1.0
  (2545, 35643)	1.0
  (2545, 35734)	1.0
  (2545, 35879)	1.0
  (2545, 36091)	1.0
  (2545, 36097)	1.0
  (2545, 36205)	1.0
  (2545, 36334)	1.0
  (2545, 36384)	1.0
  (2545, 36398)	1.0
  (2545, 36401)	7.0
  (2545, 36450)	1.0
  (2545, 36559)	2.0
  (2545, 36560)	1.0
  (2545, 36562)	4.0
  (2545, 36563)	2.0
  (2545, 36565)	4.0
  (2545, 36566)	5.0
  (2545, 36567)	1.0
  (2545, 36568)	2.0
  (2545, 36571)	2.0


In [24]:
# since the raw object is combined one, extract the raw counts for this dataset 

##### **Variables(var)**

In [25]:
#View the var of anndata and raw object

In [26]:
adata.var

""
AL627309.1
AL627309.5
AP006222.2
LINC01409
LINC01128
...
MAFIP
AC011043.1
AL592183.1
AC007325.4


In [27]:
araw.var

,gene_ids,feature_types,genome
MIR1302-2HG,ENSG00000243485,Gene Expression,GRCh38
FAM138A,ENSG00000237613,Gene Expression,GRCh38
OR4F5,ENSG00000186092,Gene Expression,GRCh38
AL627309.1,ENSG00000238009,Gene Expression,GRCh38
AL627309.3,ENSG00000239945,Gene Expression,GRCh38
...,...,...,...
AC141272.1,ENSG00000277836,Gene Expression,GRCh38
AC023491.2,ENSG00000278633,Gene Expression,GRCh38
AC007325.1,ENSG00000276017,Gene Expression,GRCh38
AC007325.4,ENSG00000278817,Gene Expression,GRCh38


In [28]:
#Ensembl IDs

In [29]:
gene_info = pd.read_table('/lustre/scratch127/cellgen/cellgeni/shibla/ref_files/2020A.gene_names.tsv')

In [30]:
gene_info

,ensembl_ids,gene,version
0,ENSG00000243485,MIR1302-2HG,2020A
1,ENSG00000237613,FAM138A,2020A
2,ENSG00000186092,OR4F5,2020A
3,ENSG00000238009,AL627309.1,2020A
4,ENSG00000239945,AL627309.3,2020A
...,...,...,...
36596,ENSG00000277836,AC141272.1,2020A
36597,ENSG00000278633,AC023491.2,2020A
36598,ENSG00000276017,AC007325.1,2020A
36599,ENSG00000278817,AC007325.4,2020A


In [31]:
#create a dictionary with gene symbols and ensembl ids from the gene information file

In [32]:
gene_info_genesym_to_ensembl = dict(zip(gene_info['gene'],gene_info['ensembl_ids']))

In [33]:
gene_info_genesym_to_ensembl

{'MIR1302-2HG': 'ENSG00000243485',
 'FAM138A': 'ENSG00000237613',
 'OR4F5': 'ENSG00000186092',
 'AL627309.1': 'ENSG00000238009',
 'AL627309.3': 'ENSG00000239945',
 'AL627309.2': 'ENSG00000239906',
 'AL627309.5': 'ENSG00000241860',
 'AL627309.4': 'ENSG00000241599',
 'AP006222.2': 'ENSG00000286448',
 'AL732372.1': 'ENSG00000236601',
 'OR4F29': 'ENSG00000284733',
 'AC114498.1': 'ENSG00000235146',
 'OR4F16': 'ENSG00000284662',
 'AL669831.2': 'ENSG00000229905',
 'LINC01409': 'ENSG00000237491',
 'FAM87B': 'ENSG00000177757',
 'LINC01128': 'ENSG00000228794',
 'LINC00115': 'ENSG00000225880',
 'FAM41C': 'ENSG00000230368',
 'AL645608.6': 'ENSG00000272438',
 'AL645608.2': 'ENSG00000230699',
 'AL645608.4': 'ENSG00000241180',
 'LINC02593': 'ENSG00000223764',
 'SAMD11': 'ENSG00000187634',
 'NOC2L': 'ENSG00000188976',
 'KLHL17': 'ENSG00000187961',
 'PLEKHN1': 'ENSG00000187583',
 'PERM1': 'ENSG00000187642',
 'AL645608.7': 'ENSG00000272512',
 'HES4': 'ENSG00000188290',
 'ISG15': 'ENSG00000187608',
 'AL6

In [34]:
#Store ensembl ids in a new column in adata.var by matching gene symbols and ensembl ids from the gene information file

In [35]:
adata.var['gene_symbols'] = adata.var_names

In [36]:
araw.var['gene_symbols'] = araw.var_names

In [37]:
araw.var_names = araw.var['gene_ids']

In [38]:
adata.var['ensembl_id'] = adata.var['gene_symbols'].map(gene_info_genesym_to_ensembl)

In [39]:
adata.var

,gene_symbols,ensembl_id
AL627309.1,AL627309.1,ENSG00000238009
AL627309.5,AL627309.5,ENSG00000241860
AP006222.2,AP006222.2,ENSG00000286448
LINC01409,LINC01409,ENSG00000237491
LINC01128,LINC01128,ENSG00000228794
...,...,...
MAFIP,MAFIP,ENSG00000274847
AC011043.1,AC011043.1,ENSG00000276256
AL592183.1,AL592183.1,ENSG00000273748
AC007325.4,AC007325.4,ENSG00000278817


In [40]:
adata.var['ensembl_id'].isna().sum()

14

In [41]:
adata.var_names = adata.var['ensembl_id']

In [42]:
adata.var

,gene_symbols,ensembl_id
ensembl_id,,
ENSG00000238009,AL627309.1,ENSG00000238009
ENSG00000241860,AL627309.5,ENSG00000241860
ENSG00000286448,AP006222.2,ENSG00000286448
ENSG00000237491,LINC01409,ENSG00000237491
ENSG00000228794,LINC01128,ENSG00000228794
...,...,...
ENSG00000274847,MAFIP,ENSG00000274847
ENSG00000276256,AC011043.1,ENSG00000276256
ENSG00000273748,AL592183.1,ENSG00000273748


In [43]:
del adata.var['gene_symbols']
del adata.var['ensembl_id']

In [44]:
del araw.var['gene_ids']
del araw.var['feature_types']
del araw.var['genome']
del araw.var['gene_symbols']

In [45]:
adata.var.index= adata.var.index.drop_duplicates()

In [46]:
adata.var

""
ensembl_id
ENSG00000238009
ENSG00000241860
ENSG00000286448
ENSG00000237491
ENSG00000228794
...
ENSG00000274847
ENSG00000276256
ENSG00000273748


In [47]:
araw.var

""
gene_ids
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277836
ENSG00000278633
ENSG00000276017


In [48]:
# load the approved genes file

In [49]:
approved_genes = pd.read_csv('/home/jovyan/CXG_DATASETS_PORTAL/gene_info/genes_approved.csv')

In [50]:
# Create a dictionary from the approved genes file using the symbols and feature id columns.

In [51]:
genedict = {key: 1 for key in list(approved_genes.feature_id)}

In [52]:
genedict

{'ERCC-00002': 1,
 'ERCC-00003': 1,
 'ERCC-00004': 1,
 'ERCC-00009': 1,
 'ERCC-00012': 1,
 'ERCC-00013': 1,
 'ERCC-00014': 1,
 'ERCC-00016': 1,
 'ERCC-00017': 1,
 'ERCC-00019': 1,
 'ERCC-00022': 1,
 'ERCC-00024': 1,
 'ERCC-00025': 1,
 'ERCC-00028': 1,
 'ERCC-00031': 1,
 'ERCC-00033': 1,
 'ERCC-00034': 1,
 'ERCC-00035': 1,
 'ERCC-00039': 1,
 'ERCC-00040': 1,
 'ERCC-00041': 1,
 'ERCC-00042': 1,
 'ERCC-00043': 1,
 'ERCC-00044': 1,
 'ERCC-00046': 1,
 'ERCC-00048': 1,
 'ERCC-00051': 1,
 'ERCC-00053': 1,
 'ERCC-00054': 1,
 'ERCC-00057': 1,
 'ERCC-00058': 1,
 'ERCC-00059': 1,
 'ERCC-00060': 1,
 'ERCC-00061': 1,
 'ERCC-00062': 1,
 'ERCC-00067': 1,
 'ERCC-00069': 1,
 'ERCC-00071': 1,
 'ERCC-00073': 1,
 'ERCC-00074': 1,
 'ERCC-00075': 1,
 'ERCC-00076': 1,
 'ERCC-00077': 1,
 'ERCC-00078': 1,
 'ERCC-00079': 1,
 'ERCC-00081': 1,
 'ERCC-00083': 1,
 'ERCC-00084': 1,
 'ERCC-00085': 1,
 'ERCC-00086': 1,
 'ERCC-00092': 1,
 'ERCC-00095': 1,
 'ERCC-00096': 1,
 'ERCC-00097': 1,
 'ERCC-00098': 1,
 'ERCC-000

In [53]:
len(genedict)

116184

In [54]:
# Filter out the genes which are not in the approved genes file

In [55]:
var_to_keep_adata = [x for x in adata.var_names if (x in genedict)]
var_to_keep_araw = [x for x in araw.var_names if (x in genedict)]

In [56]:
len(var_to_keep_adata)

17900

In [57]:
len(var_to_keep_araw)

36503

In [58]:
# Modify the anndata object by filtering out the filtered genes. copy the index column values to a new column called gene_symbols

In [59]:
adata = adata[:, var_to_keep_adata].copy()
araw = araw[:, var_to_keep_araw].copy()

In [60]:
#  View the var

In [61]:
adata.var

""
ensembl_id
ENSG00000238009
ENSG00000241860
ENSG00000286448
ENSG00000237491
ENSG00000228794
...
ENSG00000274847
ENSG00000276256
ENSG00000273748


In [62]:
araw.var

""
gene_ids
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277836
ENSG00000278633
ENSG00000276017


feature is filtered

In [63]:
def add_zero():
	global adata
	global araw
	if araw.shape[1] > adata.shape[1]:
		genes_add = [x for x in araw.var.index.to_list() if x not in adata.var.index.to_list()]
		new_matrix = sparse.csr_matrix((adata.X.data, adata.X.indices, adata.X.indptr), shape = araw.shape)
		all_genes = adata.var.index.to_list()
		all_genes.extend(genes_add)
		new_var = pd.DataFrame(index=all_genes)
		new_var = pd.merge(new_var, araw.var, left_index=True, right_index=True, how='left')
		new_var['feature_is_filtered'] = False
		new_var.loc[genes_add, 'feature_is_filtered'] = True
		new_adata = ad.AnnData(X=new_matrix, obs=adata.obs, var=new_var, uns=adata.uns, obsm=adata.obsm)
		if adata.layers:
			for layer in adata.layers:
				new_layer = sparse.csr_matrix((adata.layers[layer].data, adata.layers[layer].indices, adata.layers[layer].indptr), shape = araw.shape)
				new_adata.layers[layer] = new_layer
		new_adata = new_adata[:,araw.var.index.to_list()]
		new_adata.var = new_adata.var.merge(adata.var, left_index=True, right_index=True, how='left')
		adata = new_adata
	else:
		adata.var['feature_is_filtered'] = False

In [64]:
add_zero()

In [65]:
# view var

In [66]:
adata.var

,feature_is_filtered
ENSG00000243485,True
ENSG00000237613,True
ENSG00000186092,True
ENSG00000238009,False
ENSG00000239945,True
...,...
ENSG00000277836,True
ENSG00000278633,True
ENSG00000276017,True
ENSG00000278817,False


In [67]:
list(adata.var['feature_is_filtered'].unique())

[True, False]

In [68]:
True_count = (adata.var['feature_is_filtered']== True).sum()

In [69]:
True_count

18603

In [70]:
araw.var

""
gene_ids
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277836
ENSG00000278633
ENSG00000276017


#### **Observations(obs) (Cell metadata)**

In [71]:
#view obs

In [72]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_PTGDS+ fibroblasts,c2l_MigDC,c2l_Neuronal_SchwannC,c2l_SMC,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,578,790,21,2.658228,144,18.227848,AAACAAGTATCTCCCA-1,...,0.038756,0.027551,0.012832,0.036554,0.699707,5,4.319368,0.860222,0.014872,0.087485
AAACAGAGCGACTCCT-1,1,14,94,396,509,21,4.125737,66,12.966601,AAACAGAGCGACTCCT-1,...,0.021569,0.001684,0.016771,0.122106,0.005634,1,0.267057,0.531281,0.016486,0.012198
AAACATTTCCCGGATT-1,1,61,97,833,1406,46,3.271693,302,21.479374,AAACATTTCCCGGATT-1,...,0.022693,0.009242,0.012962,0.028560,0.891764,5,9.355833,0.920093,0.024227,0.017182
AAACCCGAACGAAATC-1,1,45,115,922,1528,55,3.599476,272,17.801048,AAACCCGAACGAAATC-1,...,0.034072,0.003741,0.020218,0.027309,0.618153,0,5.064902,0.396004,0.037257,0.030735
AAACCGGAAATGTTAA-1,1,54,124,920,1497,102,6.813627,317,21.175684,AAACCGGAAATGTTAA-1,...,0.021015,0.008656,0.018985,0.031870,0.973554,4,10.146673,1.579100,0.065374,0.013453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,22,42,333,429,28,6.526806,57,13.286713,TTGTTAGCAAATTCGA-1,...,0.008725,0.000228,0.014530,0.044013,0.012872,3,0.159194,0.855043,0.037624,0.002463
TTGTTCAGTGTGCTAC-1,1,24,64,1032,1744,183,10.493119,289,16.571102,TTGTTCAGTGTGCTAC-1,...,0.015111,0.003726,0.066904,0.155799,0.670384,4,4.183495,1.241941,0.344626,0.014597
TTGTTGTGTGTCAAGA-1,1,31,77,2008,4265,309,7.245018,665,15.592029,TTGTTGTGTGTCAAGA-1,...,0.062274,0.007115,0.057018,0.217564,0.538156,4,4.006926,6.600325,0.351264,0.022195


In [73]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1'],
      dtype='object')

#### **assay_ontology_term_id**

In [74]:
# add the assay_ontology_term_id column

In [75]:
adata.obs['assay_ontology_term_id'] = ['EFO:0010961'] * len(adata.obs)

In [76]:
# change datatype of the column

In [77]:
adata.obs['assay_ontology_term_id'] = adata.obs['assay_ontology_term_id'].astype('category')

In [78]:
# view adata.obs

In [79]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_MigDC,c2l_Neuronal_SchwannC,c2l_SMC,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,578,790,21,2.658228,144,18.227848,AAACAAGTATCTCCCA-1,...,0.027551,0.012832,0.036554,0.699707,5,4.319368,0.860222,0.014872,0.087485,EFO:0010961
AAACAGAGCGACTCCT-1,1,14,94,396,509,21,4.125737,66,12.966601,AAACAGAGCGACTCCT-1,...,0.001684,0.016771,0.122106,0.005634,1,0.267057,0.531281,0.016486,0.012198,EFO:0010961
AAACATTTCCCGGATT-1,1,61,97,833,1406,46,3.271693,302,21.479374,AAACATTTCCCGGATT-1,...,0.009242,0.012962,0.028560,0.891764,5,9.355833,0.920093,0.024227,0.017182,EFO:0010961
AAACCCGAACGAAATC-1,1,45,115,922,1528,55,3.599476,272,17.801048,AAACCCGAACGAAATC-1,...,0.003741,0.020218,0.027309,0.618153,0,5.064902,0.396004,0.037257,0.030735,EFO:0010961
AAACCGGAAATGTTAA-1,1,54,124,920,1497,102,6.813627,317,21.175684,AAACCGGAAATGTTAA-1,...,0.008656,0.018985,0.031870,0.973554,4,10.146673,1.579100,0.065374,0.013453,EFO:0010961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,22,42,333,429,28,6.526806,57,13.286713,TTGTTAGCAAATTCGA-1,...,0.000228,0.014530,0.044013,0.012872,3,0.159194,0.855043,0.037624,0.002463,EFO:0010961
TTGTTCAGTGTGCTAC-1,1,24,64,1032,1744,183,10.493119,289,16.571102,TTGTTCAGTGTGCTAC-1,...,0.003726,0.066904,0.155799,0.670384,4,4.183495,1.241941,0.344626,0.014597,EFO:0010961
TTGTTGTGTGTCAAGA-1,1,31,77,2008,4265,309,7.245018,665,15.592029,TTGTTGTGTGTCAAGA-1,...,0.007115,0.057018,0.217564,0.538156,4,4.006926,6.600325,0.351264,0.022195,EFO:0010961


#### **cell_type_ontology_term_id**

In [80]:
#get the column in adata.obs related. to cell type annotation

In [81]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id'],
      dtype='object')

In [82]:
adata.obsm

AxisArrays with keys: X_spatial

In [83]:
c2l_columns = [col for col in adata.obs.columns if col.startswith('c2l')]

In [84]:
c2l_columns

['c2l_Th',
 'c2l_NK',
 'c2l_APOD+ fibroblasts',
 'c2l_CD8+ T RM',
 'c2l_T reg',
 'c2l_Macro1_2',
 'c2l_DC1',
 'c2l_SFRP2+ fibroblasts',
 'c2l_TAGLN+ pericytes',
 'c2l_POSTN+ fibroblasts',
 'c2l_RGS5+ pericytes',
 'c2l_VEC',
 'c2l_Tc',
 'c2l_ILC_NK',
 'c2l_BC',
 'c2l_Monocytes',
 'c2l_MastC',
 'c2l_Melanocytes',
 'c2l_DC2',
 'c2l_LEC',
 'c2l_PlasmaC',
 'c2l_PTGDS+ fibroblasts',
 'c2l_MigDC',
 'c2l_Neuronal_SchwannC',
 'c2l_SMC',
 'c2l_Skeletal muscle cells',
 'c2l_Suprabasal keratinocytes',
 'c2l_Basal keratinocytes',
 'c2l_Chondrocytes',
 'c2l_IL8+ DC1']

In [85]:
adata.obs['max_c2l_column'] = adata.obs[c2l_columns].idxmax(axis=1)

In [86]:
adata.obs['max_c2l_column_value'] = adata.obs[c2l_columns].max(axis=1)

In [87]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_SMC,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,578,790,21,2.658228,144,18.227848,AAACAAGTATCTCCCA-1,...,0.036554,0.699707,5,4.319368,0.860222,0.014872,0.087485,EFO:0010961,c2l_Suprabasal keratinocytes,4.319368
AAACAGAGCGACTCCT-1,1,14,94,396,509,21,4.125737,66,12.966601,AAACAGAGCGACTCCT-1,...,0.122106,0.005634,1,0.267057,0.531281,0.016486,0.012198,EFO:0010961,c2l_SFRP2+ fibroblasts,5.249287
AAACATTTCCCGGATT-1,1,61,97,833,1406,46,3.271693,302,21.479374,AAACATTTCCCGGATT-1,...,0.028560,0.891764,5,9.355833,0.920093,0.024227,0.017182,EFO:0010961,c2l_Suprabasal keratinocytes,9.355833
AAACCCGAACGAAATC-1,1,45,115,922,1528,55,3.599476,272,17.801048,AAACCCGAACGAAATC-1,...,0.027309,0.618153,0,5.064902,0.396004,0.037257,0.030735,EFO:0010961,c2l_SFRP2+ fibroblasts,7.974823
AAACCGGAAATGTTAA-1,1,54,124,920,1497,102,6.813627,317,21.175684,AAACCGGAAATGTTAA-1,...,0.031870,0.973554,4,10.146673,1.579100,0.065374,0.013453,EFO:0010961,c2l_Suprabasal keratinocytes,10.146673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,22,42,333,429,28,6.526806,57,13.286713,TTGTTAGCAAATTCGA-1,...,0.044013,0.012872,3,0.159194,0.855043,0.037624,0.002463,EFO:0010961,c2l_SFRP2+ fibroblasts,3.812920
TTGTTCAGTGTGCTAC-1,1,24,64,1032,1744,183,10.493119,289,16.571102,TTGTTCAGTGTGCTAC-1,...,0.155799,0.670384,4,4.183495,1.241941,0.344626,0.014597,EFO:0010961,c2l_Suprabasal keratinocytes,4.183495
TTGTTGTGTGTCAAGA-1,1,31,77,2008,4265,309,7.245018,665,15.592029,TTGTTGTGTGTCAAGA-1,...,0.217564,0.538156,4,4.006926,6.600325,0.351264,0.022195,EFO:0010961,c2l_SFRP2+ fibroblasts,9.262947


In [88]:
mapping= {'c2l_Th' :'CL:0000912',
 'c2l_NK':'CL:0000623',
 'c2l_APOD+ fibroblasts':'CL:0000057',
 'c2l_CD8+ T RM':'CL:0001203' ,
 'c2l_T reg':'CL:0000815',
 'c2l_Macro1_2':'CL:0000235',
 'c2l_DC1':'CL:0000990',
 'c2l_SFRP2+ fibroblasts':'CL:0000057',
 'c2l_TAGLN+ pericytes':'CL:0000669',
 'c2l_POSTN+ fibroblasts':'CL:0000057',
 'c2l_RGS5+ pericytes':'CL:0000669',
 'c2l_VEC':'CL:0002139',
 'c2l_Tc':'CL:0000910',
 'c2l_ILC_NK':'CL:0001065', #not NK
 'c2l_BC':'CL:0000646',
 'c2l_Monocytes':'CL:0000576',
 'c2l_MastC':'CL:0000097',
 'c2l_Melanocytes':'CL:1000458',
 'c2l_DC2':'CL:0000784',
 'c2l_LEC':'CL:0002138',
 'c2l_PlasmaC':'CL:0000786',
 'c2l_PTGDS+ fibroblasts':'CL:0000057',
 'c2l_MigDC':'CL:0000451', #not mig
 'c2l_Neuronal_SchwannC':'CL:0002573',
 'c2l_SMC':'CL:0000192',
 'c2l_Skeletal muscle cells':'CL:0000188',
 'c2l_Suprabasal keratinocytes':'CL:4033013',
 'c2l_Basal keratinocytes':'CL:0002187',
 'c2l_Chondrocytes':'CL:0000138',
 'c2l_IL8+ DC1':'CL:0000990'}

In [89]:
# create a dictionary of cell type and ontology term

In [90]:
# add the cell_type_ontology_term_id column

In [91]:
adata.obs['cell_type_ontology_term_id'] = adata.obs['max_c2l_column'].map(mapping)

In [92]:
# change datatype of the column

In [93]:
adata.obs['cell_type_ontology_term_id'] = adata.obs['cell_type_ontology_term_id'].astype('category')

In [94]:
list(adata.obs['cell_type_ontology_term_id'].unique())

['CL:4033013',
 'CL:0000057',
 'CL:0002187',
 'CL:0002139',
 'CL:0000669',
 'CL:0002138',
 'CL:0000192',
 'CL:0000646',
 'CL:0000235',
 'CL:0000097',
 'CL:0000784']

In [95]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,578,790,21,2.658228,144,18.227848,AAACAAGTATCTCCCA-1,...,0.699707,5,4.319368,0.860222,0.014872,0.087485,EFO:0010961,c2l_Suprabasal keratinocytes,4.319368,CL:4033013
AAACAGAGCGACTCCT-1,1,14,94,396,509,21,4.125737,66,12.966601,AAACAGAGCGACTCCT-1,...,0.005634,1,0.267057,0.531281,0.016486,0.012198,EFO:0010961,c2l_SFRP2+ fibroblasts,5.249287,CL:0000057
AAACATTTCCCGGATT-1,1,61,97,833,1406,46,3.271693,302,21.479374,AAACATTTCCCGGATT-1,...,0.891764,5,9.355833,0.920093,0.024227,0.017182,EFO:0010961,c2l_Suprabasal keratinocytes,9.355833,CL:4033013
AAACCCGAACGAAATC-1,1,45,115,922,1528,55,3.599476,272,17.801048,AAACCCGAACGAAATC-1,...,0.618153,0,5.064902,0.396004,0.037257,0.030735,EFO:0010961,c2l_SFRP2+ fibroblasts,7.974823,CL:0000057
AAACCGGAAATGTTAA-1,1,54,124,920,1497,102,6.813627,317,21.175684,AAACCGGAAATGTTAA-1,...,0.973554,4,10.146673,1.579100,0.065374,0.013453,EFO:0010961,c2l_Suprabasal keratinocytes,10.146673,CL:4033013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,22,42,333,429,28,6.526806,57,13.286713,TTGTTAGCAAATTCGA-1,...,0.012872,3,0.159194,0.855043,0.037624,0.002463,EFO:0010961,c2l_SFRP2+ fibroblasts,3.812920,CL:0000057
TTGTTCAGTGTGCTAC-1,1,24,64,1032,1744,183,10.493119,289,16.571102,TTGTTCAGTGTGCTAC-1,...,0.670384,4,4.183495,1.241941,0.344626,0.014597,EFO:0010961,c2l_Suprabasal keratinocytes,4.183495,CL:4033013
TTGTTGTGTGTCAAGA-1,1,31,77,2008,4265,309,7.245018,665,15.592029,TTGTTGTGTGTCAAGA-1,...,0.538156,4,4.006926,6.600325,0.351264,0.022195,EFO:0010961,c2l_SFRP2+ fibroblasts,9.262947,CL:0000057


In [96]:
araw.obs

""
AAACAAGTATCTCCCA-1
AAACAGAGCGACTCCT-1
AAACATTTCCCGGATT-1
AAACCCGAACGAAATC-1
AAACCGGAAATGTTAA-1
...
TTGTTAGCAAATTCGA-1
TTGTTCAGTGTGCTAC-1
TTGTTGTGTGTCAAGA-1
TTGTTTCACATCCAGG-1


#### **donor_id**

In [97]:
#identify the column in adata.obs which provides donor information

In [98]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'max_c2l_column', 'max_c2l_column_value',
       'cell_type_ontology_term_id'],
      dtype='object')

In [99]:
# add the donor_id column

In [100]:
adata.obs['donor_id'] = ['WSSKNKCLsp12887264'] * len(adata.obs)

In [101]:
# change datatype of the column

In [102]:
adata.obs['donor_id'] = adata.obs['donor_id'].astype('category')

In [103]:
# view unique values of donor_id column

In [104]:
list(adata.obs['donor_id'].unique())

['WSSKNKCLsp12887264']

In [105]:
#view obs

In [106]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,578,790,21,2.658228,144,18.227848,AAACAAGTATCTCCCA-1,...,5,4.319368,0.860222,0.014872,0.087485,EFO:0010961,c2l_Suprabasal keratinocytes,4.319368,CL:4033013,WSSKNKCLsp12887264
AAACAGAGCGACTCCT-1,1,14,94,396,509,21,4.125737,66,12.966601,AAACAGAGCGACTCCT-1,...,1,0.267057,0.531281,0.016486,0.012198,EFO:0010961,c2l_SFRP2+ fibroblasts,5.249287,CL:0000057,WSSKNKCLsp12887264
AAACATTTCCCGGATT-1,1,61,97,833,1406,46,3.271693,302,21.479374,AAACATTTCCCGGATT-1,...,5,9.355833,0.920093,0.024227,0.017182,EFO:0010961,c2l_Suprabasal keratinocytes,9.355833,CL:4033013,WSSKNKCLsp12887264
AAACCCGAACGAAATC-1,1,45,115,922,1528,55,3.599476,272,17.801048,AAACCCGAACGAAATC-1,...,0,5.064902,0.396004,0.037257,0.030735,EFO:0010961,c2l_SFRP2+ fibroblasts,7.974823,CL:0000057,WSSKNKCLsp12887264
AAACCGGAAATGTTAA-1,1,54,124,920,1497,102,6.813627,317,21.175684,AAACCGGAAATGTTAA-1,...,4,10.146673,1.579100,0.065374,0.013453,EFO:0010961,c2l_Suprabasal keratinocytes,10.146673,CL:4033013,WSSKNKCLsp12887264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,22,42,333,429,28,6.526806,57,13.286713,TTGTTAGCAAATTCGA-1,...,3,0.159194,0.855043,0.037624,0.002463,EFO:0010961,c2l_SFRP2+ fibroblasts,3.812920,CL:0000057,WSSKNKCLsp12887264
TTGTTCAGTGTGCTAC-1,1,24,64,1032,1744,183,10.493119,289,16.571102,TTGTTCAGTGTGCTAC-1,...,4,4.183495,1.241941,0.344626,0.014597,EFO:0010961,c2l_Suprabasal keratinocytes,4.183495,CL:4033013,WSSKNKCLsp12887264
TTGTTGTGTGTCAAGA-1,1,31,77,2008,4265,309,7.245018,665,15.592029,TTGTTGTGTGTCAAGA-1,...,4,4.006926,6.600325,0.351264,0.022195,EFO:0010961,c2l_SFRP2+ fibroblasts,9.262947,CL:0000057,WSSKNKCLsp12887264


In [107]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'max_c2l_column', 'max_c2l_column_value',
       'cell_type_ontology_term_id', 'donor_id'],
      dtype='object')

#### **development_stage_ontology_term_id**

In [108]:
# identify the column in adata which corresponds to age

In [109]:
# add the development_stage_ontology_term_id column

In [110]:
suppl_info = pd.read_csv('/home/jovyan/CXG_DATASETS_PORTAL/Spatial_skin_atlas/spatial/suppl_info_spatialskin.csv')

In [111]:
mapping = dict(zip(suppl_info['donor'], suppl_info['development_stage_ontology_term_id']))

In [112]:
adata.obs['development_stage_ontology_term_id'] = adata.obs['donor_id'].map(mapping)

In [113]:
# change datatype of the column

In [114]:
adata.obs['development_stage_ontology_term_id'] = adata.obs['development_stage_ontology_term_id'].astype('category')

In [115]:
# view unique values of development_stage_ontology_term_id column

In [116]:
list(adata.obs['development_stage_ontology_term_id'].unique())

['HsapDv:0000154']

In [117]:
# view adata.obs

In [118]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,578,790,21,2.658228,144,18.227848,AAACAAGTATCTCCCA-1,...,4.319368,0.860222,0.014872,0.087485,EFO:0010961,c2l_Suprabasal keratinocytes,4.319368,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154
AAACAGAGCGACTCCT-1,1,14,94,396,509,21,4.125737,66,12.966601,AAACAGAGCGACTCCT-1,...,0.267057,0.531281,0.016486,0.012198,EFO:0010961,c2l_SFRP2+ fibroblasts,5.249287,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154
AAACATTTCCCGGATT-1,1,61,97,833,1406,46,3.271693,302,21.479374,AAACATTTCCCGGATT-1,...,9.355833,0.920093,0.024227,0.017182,EFO:0010961,c2l_Suprabasal keratinocytes,9.355833,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154
AAACCCGAACGAAATC-1,1,45,115,922,1528,55,3.599476,272,17.801048,AAACCCGAACGAAATC-1,...,5.064902,0.396004,0.037257,0.030735,EFO:0010961,c2l_SFRP2+ fibroblasts,7.974823,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154
AAACCGGAAATGTTAA-1,1,54,124,920,1497,102,6.813627,317,21.175684,AAACCGGAAATGTTAA-1,...,10.146673,1.579100,0.065374,0.013453,EFO:0010961,c2l_Suprabasal keratinocytes,10.146673,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,22,42,333,429,28,6.526806,57,13.286713,TTGTTAGCAAATTCGA-1,...,0.159194,0.855043,0.037624,0.002463,EFO:0010961,c2l_SFRP2+ fibroblasts,3.812920,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154
TTGTTCAGTGTGCTAC-1,1,24,64,1032,1744,183,10.493119,289,16.571102,TTGTTCAGTGTGCTAC-1,...,4.183495,1.241941,0.344626,0.014597,EFO:0010961,c2l_Suprabasal keratinocytes,4.183495,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154
TTGTTGTGTGTCAAGA-1,1,31,77,2008,4265,309,7.245018,665,15.592029,TTGTTGTGTGTCAAGA-1,...,4.006926,6.600325,0.351264,0.022195,EFO:0010961,c2l_SFRP2+ fibroblasts,9.262947,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154


#### **disease_ontology_term_id**

In [119]:
# Assign normal since all are healthy patients

In [120]:
# add the disease_ontology_term_id column

In [121]:
suppl_info = pd.read_csv('/home/jovyan/CXG_DATASETS_PORTAL/Spatial_skin_atlas/spatial/suppl_info_spatialskin.csv')

In [122]:
mapping = dict(zip(suppl_info['donor'], suppl_info['disease_ontology_term_id']))

In [123]:
adata.obs['disease_ontology_term_id'] = adata.obs['donor_id'].map(mapping)

In [124]:
#change data type of column

In [125]:
adata.obs['disease_ontology_term_id'] = adata.obs['disease_ontology_term_id'].astype('category')

In [126]:
# view obs

In [127]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,578,790,21,2.658228,144,18.227848,AAACAAGTATCTCCCA-1,...,0.860222,0.014872,0.087485,EFO:0010961,c2l_Suprabasal keratinocytes,4.319368,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461
AAACAGAGCGACTCCT-1,1,14,94,396,509,21,4.125737,66,12.966601,AAACAGAGCGACTCCT-1,...,0.531281,0.016486,0.012198,EFO:0010961,c2l_SFRP2+ fibroblasts,5.249287,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461
AAACATTTCCCGGATT-1,1,61,97,833,1406,46,3.271693,302,21.479374,AAACATTTCCCGGATT-1,...,0.920093,0.024227,0.017182,EFO:0010961,c2l_Suprabasal keratinocytes,9.355833,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461
AAACCCGAACGAAATC-1,1,45,115,922,1528,55,3.599476,272,17.801048,AAACCCGAACGAAATC-1,...,0.396004,0.037257,0.030735,EFO:0010961,c2l_SFRP2+ fibroblasts,7.974823,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461
AAACCGGAAATGTTAA-1,1,54,124,920,1497,102,6.813627,317,21.175684,AAACCGGAAATGTTAA-1,...,1.579100,0.065374,0.013453,EFO:0010961,c2l_Suprabasal keratinocytes,10.146673,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,22,42,333,429,28,6.526806,57,13.286713,TTGTTAGCAAATTCGA-1,...,0.855043,0.037624,0.002463,EFO:0010961,c2l_SFRP2+ fibroblasts,3.812920,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461
TTGTTCAGTGTGCTAC-1,1,24,64,1032,1744,183,10.493119,289,16.571102,TTGTTCAGTGTGCTAC-1,...,1.241941,0.344626,0.014597,EFO:0010961,c2l_Suprabasal keratinocytes,4.183495,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461
TTGTTGTGTGTCAAGA-1,1,31,77,2008,4265,309,7.245018,665,15.592029,TTGTTGTGTGTCAAGA-1,...,6.600325,0.351264,0.022195,EFO:0010961,c2l_SFRP2+ fibroblasts,9.262947,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461


#### **is_primary_data**

In [128]:
adata.obs['is_primary_data'] = [True] * len(adata.obs)

In [129]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,578,790,21,2.658228,144,18.227848,AAACAAGTATCTCCCA-1,...,0.014872,0.087485,EFO:0010961,c2l_Suprabasal keratinocytes,4.319368,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True
AAACAGAGCGACTCCT-1,1,14,94,396,509,21,4.125737,66,12.966601,AAACAGAGCGACTCCT-1,...,0.016486,0.012198,EFO:0010961,c2l_SFRP2+ fibroblasts,5.249287,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True
AAACATTTCCCGGATT-1,1,61,97,833,1406,46,3.271693,302,21.479374,AAACATTTCCCGGATT-1,...,0.024227,0.017182,EFO:0010961,c2l_Suprabasal keratinocytes,9.355833,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True
AAACCCGAACGAAATC-1,1,45,115,922,1528,55,3.599476,272,17.801048,AAACCCGAACGAAATC-1,...,0.037257,0.030735,EFO:0010961,c2l_SFRP2+ fibroblasts,7.974823,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True
AAACCGGAAATGTTAA-1,1,54,124,920,1497,102,6.813627,317,21.175684,AAACCGGAAATGTTAA-1,...,0.065374,0.013453,EFO:0010961,c2l_Suprabasal keratinocytes,10.146673,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,22,42,333,429,28,6.526806,57,13.286713,TTGTTAGCAAATTCGA-1,...,0.037624,0.002463,EFO:0010961,c2l_SFRP2+ fibroblasts,3.812920,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True
TTGTTCAGTGTGCTAC-1,1,24,64,1032,1744,183,10.493119,289,16.571102,TTGTTCAGTGTGCTAC-1,...,0.344626,0.014597,EFO:0010961,c2l_Suprabasal keratinocytes,4.183495,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True
TTGTTGTGTGTCAAGA-1,1,31,77,2008,4265,309,7.245018,665,15.592029,TTGTTGTGTGTCAAGA-1,...,0.351264,0.022195,EFO:0010961,c2l_SFRP2+ fibroblasts,9.262947,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True


In [130]:
#change data type of column

In [131]:
adata.obs['is_primary_data'] = adata.obs['is_primary_data'].astype('bool')

#### **organism_ontology_term_id**

In [132]:
# assign organism id 

In [133]:
adata.obs['organism_ontology_term_id'] = ['NCBITaxon:9606'] * len(adata.obs)

In [134]:
#change data type of column

In [135]:
adata.obs['organism_ontology_term_id'] = adata.obs['organism_ontology_term_id'].astype('category')

In [136]:
# view obs

In [137]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,578,790,21,2.658228,144,18.227848,AAACAAGTATCTCCCA-1,...,0.087485,EFO:0010961,c2l_Suprabasal keratinocytes,4.319368,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606
AAACAGAGCGACTCCT-1,1,14,94,396,509,21,4.125737,66,12.966601,AAACAGAGCGACTCCT-1,...,0.012198,EFO:0010961,c2l_SFRP2+ fibroblasts,5.249287,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606
AAACATTTCCCGGATT-1,1,61,97,833,1406,46,3.271693,302,21.479374,AAACATTTCCCGGATT-1,...,0.017182,EFO:0010961,c2l_Suprabasal keratinocytes,9.355833,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606
AAACCCGAACGAAATC-1,1,45,115,922,1528,55,3.599476,272,17.801048,AAACCCGAACGAAATC-1,...,0.030735,EFO:0010961,c2l_SFRP2+ fibroblasts,7.974823,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606
AAACCGGAAATGTTAA-1,1,54,124,920,1497,102,6.813627,317,21.175684,AAACCGGAAATGTTAA-1,...,0.013453,EFO:0010961,c2l_Suprabasal keratinocytes,10.146673,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,22,42,333,429,28,6.526806,57,13.286713,TTGTTAGCAAATTCGA-1,...,0.002463,EFO:0010961,c2l_SFRP2+ fibroblasts,3.812920,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606
TTGTTCAGTGTGCTAC-1,1,24,64,1032,1744,183,10.493119,289,16.571102,TTGTTCAGTGTGCTAC-1,...,0.014597,EFO:0010961,c2l_Suprabasal keratinocytes,4.183495,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606
TTGTTGTGTGTCAAGA-1,1,31,77,2008,4265,309,7.245018,665,15.592029,TTGTTGTGTGTCAAGA-1,...,0.022195,EFO:0010961,c2l_SFRP2+ fibroblasts,9.262947,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606


#### **self_reported_ethnicity_ontology_term_id**

In [138]:
adata.obs['self_reported_ethnicity_ontology_term_id'] = ['unknown'] * len(adata.obs)

In [139]:
# change data type

In [140]:
adata.obs['self_reported_ethnicity_ontology_term_id'] = adata.obs['self_reported_ethnicity_ontology_term_id'].astype('category')

In [141]:
# view obs

In [142]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,578,790,21,2.658228,144,18.227848,AAACAAGTATCTCCCA-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,4.319368,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown
AAACAGAGCGACTCCT-1,1,14,94,396,509,21,4.125737,66,12.966601,AAACAGAGCGACTCCT-1,...,EFO:0010961,c2l_SFRP2+ fibroblasts,5.249287,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown
AAACATTTCCCGGATT-1,1,61,97,833,1406,46,3.271693,302,21.479374,AAACATTTCCCGGATT-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,9.355833,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown
AAACCCGAACGAAATC-1,1,45,115,922,1528,55,3.599476,272,17.801048,AAACCCGAACGAAATC-1,...,EFO:0010961,c2l_SFRP2+ fibroblasts,7.974823,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown
AAACCGGAAATGTTAA-1,1,54,124,920,1497,102,6.813627,317,21.175684,AAACCGGAAATGTTAA-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,10.146673,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,22,42,333,429,28,6.526806,57,13.286713,TTGTTAGCAAATTCGA-1,...,EFO:0010961,c2l_SFRP2+ fibroblasts,3.812920,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown
TTGTTCAGTGTGCTAC-1,1,24,64,1032,1744,183,10.493119,289,16.571102,TTGTTCAGTGTGCTAC-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,4.183495,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown
TTGTTGTGTGTCAAGA-1,1,31,77,2008,4265,309,7.245018,665,15.592029,TTGTTGTGTGTCAAGA-1,...,EFO:0010961,c2l_SFRP2+ fibroblasts,9.262947,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown


In [143]:
list(adata.obs['self_reported_ethnicity_ontology_term_id'].unique())

['unknown']

#### **sex_ontology_term_id**

In [144]:
suppl_info = pd.read_csv('/home/jovyan/CXG_DATASETS_PORTAL/Spatial_skin_atlas/spatial/suppl_info_spatialskin.csv')

In [145]:
mapping = dict(zip(suppl_info['donor'], suppl_info['sex_ontology_term_id']))

In [146]:
adata.obs['sex_ontology_term_id'] = adata.obs['donor_id'].map(mapping)

In [147]:
# change data type

In [148]:
adata.obs['sex_ontology_term_id'] = adata.obs['sex_ontology_term_id'].astype('category')

In [149]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,578,790,21,2.658228,144,18.227848,AAACAAGTATCTCCCA-1,...,c2l_Suprabasal keratinocytes,4.319368,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
AAACAGAGCGACTCCT-1,1,14,94,396,509,21,4.125737,66,12.966601,AAACAGAGCGACTCCT-1,...,c2l_SFRP2+ fibroblasts,5.249287,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
AAACATTTCCCGGATT-1,1,61,97,833,1406,46,3.271693,302,21.479374,AAACATTTCCCGGATT-1,...,c2l_Suprabasal keratinocytes,9.355833,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
AAACCCGAACGAAATC-1,1,45,115,922,1528,55,3.599476,272,17.801048,AAACCCGAACGAAATC-1,...,c2l_SFRP2+ fibroblasts,7.974823,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
AAACCGGAAATGTTAA-1,1,54,124,920,1497,102,6.813627,317,21.175684,AAACCGGAAATGTTAA-1,...,c2l_Suprabasal keratinocytes,10.146673,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,22,42,333,429,28,6.526806,57,13.286713,TTGTTAGCAAATTCGA-1,...,c2l_SFRP2+ fibroblasts,3.812920,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
TTGTTCAGTGTGCTAC-1,1,24,64,1032,1744,183,10.493119,289,16.571102,TTGTTCAGTGTGCTAC-1,...,c2l_Suprabasal keratinocytes,4.183495,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
TTGTTGTGTGTCAAGA-1,1,31,77,2008,4265,309,7.245018,665,15.592029,TTGTTGTGTGTCAAGA-1,...,c2l_SFRP2+ fibroblasts,9.262947,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384


#### **suspension_type**

In [150]:
# since visium suspension type is 'na'

In [151]:
adata.obs['suspension_type'] = ['na'] * len(adata.obs)

In [152]:
# change data type

In [153]:
adata.obs['suspension_type'] = adata.obs['suspension_type'].astype('category')

In [154]:
# view obs

In [155]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,578,790,21,2.658228,144,18.227848,AAACAAGTATCTCCCA-1,...,4.319368,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
AAACAGAGCGACTCCT-1,1,14,94,396,509,21,4.125737,66,12.966601,AAACAGAGCGACTCCT-1,...,5.249287,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
AAACATTTCCCGGATT-1,1,61,97,833,1406,46,3.271693,302,21.479374,AAACATTTCCCGGATT-1,...,9.355833,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
AAACCCGAACGAAATC-1,1,45,115,922,1528,55,3.599476,272,17.801048,AAACCCGAACGAAATC-1,...,7.974823,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
AAACCGGAAATGTTAA-1,1,54,124,920,1497,102,6.813627,317,21.175684,AAACCGGAAATGTTAA-1,...,10.146673,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,22,42,333,429,28,6.526806,57,13.286713,TTGTTAGCAAATTCGA-1,...,3.812920,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
TTGTTCAGTGTGCTAC-1,1,24,64,1032,1744,183,10.493119,289,16.571102,TTGTTCAGTGTGCTAC-1,...,4.183495,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
TTGTTGTGTGTCAAGA-1,1,31,77,2008,4265,309,7.245018,665,15.592029,TTGTTGTGTGTCAAGA-1,...,9.262947,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na


#### **tissue_ontology_term_id**

In [156]:
suppl_info = pd.read_csv('/home/jovyan/CXG_DATASETS_PORTAL/Spatial_skin_atlas/spatial/suppl_info_spatialskin.csv')

In [157]:
mapping = dict(zip(suppl_info['donor'], suppl_info['tissue_ontology_term_id']))

In [158]:
adata.obs['tissue_ontology_term_id'] = adata.obs['donor_id'].map(mapping)

In [159]:
adata.obs['tissue_ontology_term_id'] = adata.obs['tissue_ontology_term_id'].astype('category')

In [160]:
list(adata.obs['tissue_ontology_term_id'].unique())

['UBERON:0001068']

In [161]:
# view obs

In [162]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type,tissue_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,578,790,21,2.658228,144,18.227848,AAACAAGTATCTCCCA-1,...,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0001068
AAACAGAGCGACTCCT-1,1,14,94,396,509,21,4.125737,66,12.966601,AAACAGAGCGACTCCT-1,...,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0001068
AAACATTTCCCGGATT-1,1,61,97,833,1406,46,3.271693,302,21.479374,AAACATTTCCCGGATT-1,...,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0001068
AAACCCGAACGAAATC-1,1,45,115,922,1528,55,3.599476,272,17.801048,AAACCCGAACGAAATC-1,...,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0001068
AAACCGGAAATGTTAA-1,1,54,124,920,1497,102,6.813627,317,21.175684,AAACCGGAAATGTTAA-1,...,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0001068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,22,42,333,429,28,6.526806,57,13.286713,TTGTTAGCAAATTCGA-1,...,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0001068
TTGTTCAGTGTGCTAC-1,1,24,64,1032,1744,183,10.493119,289,16.571102,TTGTTCAGTGTGCTAC-1,...,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0001068
TTGTTGTGTGTCAAGA-1,1,31,77,2008,4265,309,7.245018,665,15.592029,TTGTTGTGTGTCAAGA-1,...,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0001068


In [163]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'max_c2l_column', 'max_c2l_column_value',
       'cell_type_ontology_term_id', 'donor_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'is_primary_data', 

In [164]:
del adata.obs['barcode']
del adata.obs['max_c2l_column']
del adata.obs['max_c2l_column_value']
del adata.obs['array_row']
del adata.obs['array_col']

#### **obsm (Embeddings)**

In [165]:
adata.obsm

AxisArrays with keys: X_spatial

In [166]:
adata.obsm.keys()

KeysView(AxisArrays with keys: X_spatial)

#### **uns (Dataset Metadata)**

In [167]:
adata.uns

OverloadedDict, wrapping:
	{'leiden': {'params': {'n_iterations': -1, 'random_state': 0, 'resolution': 1}}, 'log1p': {}, 'neighbors': {'connectivities_key': 'connectivities', 'distances_key': 'distances', 'params': {'method': 'umap', 'metric': 'euclidean', 'n_neighbors': 15, 'random_state': 0}}, 'pca': {'params': {'use_highly_variable': False, 'zero_center': True}, 'variance': array([390.3358   ,  89.0915   ,  57.833435 ,  48.007603 ,  34.266026 ,
        30.873213 ,  21.850403 ,  20.244635 ,  19.290384 ,  17.470387 ,
        16.876898 ,  16.069239 ,  15.737098 ,  15.515036 ,  15.374118 ,
        15.316092 ,  15.217855 ,  15.111763 ,  15.038868 ,  14.97201  ,
        14.92768  ,  14.7922735,  14.758442 ,  14.701495 ,  14.632956 ,
        14.61004  ,  14.542835 ,  14.506672 ,  14.446026 ,  14.388048 ,
        14.360164 ,  14.3192215,  14.233343 ,  14.221538 ,  14.2092495,
        14.185994 ,  14.149447 ,  14.132417 ,  14.084522 ,  14.059555 ,
        14.030101 ,  14.019683 ,  13.971034 

In [168]:
adata.uns['image_caption'] = 'Shown here is an image of 10 μm thick slice of the skin from the back stained with H&E'

In [169]:
adata.uns['title'] = 'Visium spatial - body_back1b'

In [170]:
adata.uns['default_embedding'] = 'X_spatial'

In [171]:
adata.uns.keys()

dict_keys(['leiden', 'log1p', 'neighbors', 'pca', 'spatial', 'umap', 'image_caption', 'title', 'default_embedding'])

### **Final checks and adjustments**

In [172]:
adata

AnnData object with n_obs × n_vars = 2546 × 36503
    obs: 'in_tissue', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes', 'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc', 'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes', 'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts', 'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC', 'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes', 'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'donor_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'sex_ontology_term_id', 'suspension_type', 'tissue_o

In [173]:
adata.obs.dtypes

in_tissue                                      int64
n_genes_by_counts                              int32
total_counts                                   int32
total_counts_mt                                int32
pct_counts_mt                                float32
total_counts_rb                                int32
pct_counts_rb                                float32
c2l_Th                                       float64
c2l_NK                                       float64
c2l_APOD+ fibroblasts                        float64
c2l_CD8+ T RM                                float64
c2l_T reg                                    float64
c2l_Macro1_2                                 float64
c2l_DC1                                      float64
c2l_SFRP2+ fibroblasts                       float64
c2l_TAGLN+ pericytes                         float64
c2l_POSTN+ fibroblasts                       float64
c2l_RGS5+ pericytes                          float64
c2l_VEC                                      f

In [174]:
dty = pd.DataFrame(adata.var.dtypes, columns = ['dtype'])
for c in dty[dty['dtype'] == 'float64'].index.values:
    adata.var[c] = adata.var[c].astype('float32')
    print(f"changed {c} from float64 to float32")
for c in dty[dty['dtype'] == 'int64'].index.values:
    adata.var[c] = adata.var[c].astype('int32') 
    print(f"changed {c} from int64 to int32")

In [175]:
dty = pd.DataFrame(adata.obs.dtypes, columns = ['dtype'])
for c in dty[dty['dtype'] == 'float64'].index.values:
    adata.obs[c] = adata.obs[c].astype('float32')
    print(f"changed {c} from float64 to float32")
for c in dty[dty['dtype'] == 'int64'].index.values:
    adata.obs[c] = adata.obs[c].astype('int32') 
    print(f"changed {c} from int64 to int32")
for c in dty[dty['dtype'] == 'object'].index.values:
    adata.obs[c] = adata.obs[c].astype('category') 
    print(f"changed {c} from object to category")

changed c2l_Th from float64 to float32
changed c2l_NK from float64 to float32
changed c2l_APOD+ fibroblasts from float64 to float32
changed c2l_CD8+ T RM from float64 to float32
changed c2l_T reg from float64 to float32
changed c2l_Macro1_2 from float64 to float32
changed c2l_DC1 from float64 to float32
changed c2l_SFRP2+ fibroblasts from float64 to float32
changed c2l_TAGLN+ pericytes from float64 to float32
changed c2l_POSTN+ fibroblasts from float64 to float32
changed c2l_RGS5+ pericytes from float64 to float32
changed c2l_VEC from float64 to float32
changed c2l_Tc from float64 to float32
changed c2l_ILC_NK from float64 to float32
changed c2l_BC from float64 to float32
changed c2l_Monocytes from float64 to float32
changed c2l_MastC from float64 to float32
changed c2l_Melanocytes from float64 to float32
changed c2l_DC2 from float64 to float32
changed c2l_LEC from float64 to float32
changed c2l_PlasmaC from float64 to float32
changed c2l_PTGDS+ fibroblasts from float64 to float32
chan

In [176]:
adata.obs

,in_tissue,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,c2l_Th,c2l_NK,c2l_APOD+ fibroblasts,...,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type,tissue_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,578,790,21,2.658228,144,18.227848,0.028607,0.239166,0.047835,...,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0001068
AAACAGAGCGACTCCT-1,1,396,509,21,4.125737,66,12.966601,0.006125,0.015365,0.046135,...,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0001068
AAACATTTCCCGGATT-1,1,833,1406,46,3.271693,302,21.479374,0.023438,0.019392,0.029158,...,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0001068
AAACCCGAACGAAATC-1,1,922,1528,55,3.599476,272,17.801048,0.015747,0.020670,0.044777,...,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0001068
AAACCGGAAATGTTAA-1,1,920,1497,102,6.813627,317,21.175684,0.035011,0.041792,0.039399,...,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0001068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,333,429,28,6.526806,57,13.286713,0.003090,0.005997,0.014658,...,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0001068
TTGTTCAGTGTGCTAC-1,1,1032,1744,183,10.493119,289,16.571102,0.029375,0.026716,0.255251,...,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0001068
TTGTTGTGTGTCAAGA-1,1,2008,4265,309,7.245018,665,15.592029,0.050745,0.036732,0.097633,...,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0001068


In [177]:
adata.obs.columns

Index(['in_tissue', 'n_genes_by_counts', 'total_counts', 'total_counts_mt',
       'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'c2l_Th', 'c2l_NK',
       'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2',
       'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'cell_type_ontology_term_id', 'donor_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'is_primary_data', 'organism_ontology_term_id',
       'self_reported_ethnicity_ontology_term_id', 'sex_ontology

In [178]:
adata.var

,feature_is_filtered
ENSG00000243485,True
ENSG00000237613,True
ENSG00000186092,True
ENSG00000238009,False
ENSG00000239945,True
...,...
ENSG00000277836,True
ENSG00000278633,True
ENSG00000276017,True
ENSG00000278817,False


In [179]:
adata.obs

,in_tissue,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,c2l_Th,c2l_NK,c2l_APOD+ fibroblasts,...,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type,tissue_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,578,790,21,2.658228,144,18.227848,0.028607,0.239166,0.047835,...,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0001068
AAACAGAGCGACTCCT-1,1,396,509,21,4.125737,66,12.966601,0.006125,0.015365,0.046135,...,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0001068
AAACATTTCCCGGATT-1,1,833,1406,46,3.271693,302,21.479374,0.023438,0.019392,0.029158,...,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0001068
AAACCCGAACGAAATC-1,1,922,1528,55,3.599476,272,17.801048,0.015747,0.020670,0.044777,...,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0001068
AAACCGGAAATGTTAA-1,1,920,1497,102,6.813627,317,21.175684,0.035011,0.041792,0.039399,...,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0001068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,333,429,28,6.526806,57,13.286713,0.003090,0.005997,0.014658,...,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0001068
TTGTTCAGTGTGCTAC-1,1,1032,1744,183,10.493119,289,16.571102,0.029375,0.026716,0.255251,...,CL:4033013,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0001068
TTGTTGTGTGTCAAGA-1,1,2008,4265,309,7.245018,665,15.592029,0.050745,0.036732,0.097633,...,CL:0000057,WSSKNKCLsp12887264,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0001068


In [180]:
adata.obs.columns

Index(['in_tissue', 'n_genes_by_counts', 'total_counts', 'total_counts_mt',
       'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'c2l_Th', 'c2l_NK',
       'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2',
       'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'cell_type_ontology_term_id', 'donor_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'is_primary_data', 'organism_ontology_term_id',
       'self_reported_ethnicity_ontology_term_id', 'sex_ontology

In [181]:
#check the format of expression matrix

In [182]:
adata.X

<2546x36503 sparse matrix of type '<class 'numpy.float32'>'
	with 45573400 stored elements in Compressed Sparse Row format>

In [183]:
araw.X

<2546x36503 sparse matrix of type '<class 'numpy.float32'>'
	with 3268796 stored elements in Compressed Sparse Row format>

In [184]:
#Copy raw counts to adata.raw

In [185]:
adata.raw = araw

In [186]:
#write the curated object to final_objects folder

In [187]:
adata.write('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Final_objects/body_back1b.h5ad', compression = 'gzip')